In [0]:
import numpy as np
import pickle
import pandas as pd
import re
import string
import itertools

In [0]:
data = pd.read_csv('./websitesblog/website_123weddingcards.csv')

In [0]:
data['text'].head()

0    Your Account,Access account and orders,\r\n\t\...
1    Your Account,Access account and orders,\r\n\t\...
2    Your Account,Access account and orders,Shop Cu...
3    Your Account,Access account and orders,Shop Cu...
4    Your Account,Access account and orders,SUBMIT,...
Name: text, dtype: object

In [0]:
lists_data = [re.sub(r'[^\x00-\x7f]','',re.sub('[\t\r\n,)([\]\d!%|!#$%&*+,.-/:;<=>?@^_`{|}~]',' ',str(i))) for i in data['text']]

In [0]:
lists_data[2]

'Your Account Access account and orders Shop Custom Cards      For the people you Love     New  Arrivals  Browse affordable wedding Invitations        Quality Guaranteed  Pick up the unique invitations   Push your  celebration to a next level  Announce Your Big Day in High Style   Sample order amount fully adjustable on bulk orders  Create an invitation of your dreams  Customize Now  Add Ons that enhance your wedding invitations  Timothy Anderson United States   Feb   th       Rx John Austria   Feb   nd       Sam Adrew Belgium   Feb   st       Jack  MacLeod United States   Jan   th       Harley Australia   Jan   nd       Liam Tucker United States   Dec   rd       Selecting a card for your wedding is one of the difficult tasks for a bride as well as the groom  So     WeddingCards is here to help you with a wide range of online wedding cards with memorizing patterns  motifs  and designs  Our limitless wedding invites are unique  affordable and perfect for your special occasion     Weddin

In [0]:
with open('spelling_data_pruned.obj', 'rb') as f:
    training_data = pickle.load(f)

In [0]:
import re
# train_texts = [s.lower() for s in training_data]
train_texts = [re.sub(r'[^\x00-\x7f]','',re.sub('[\t\r\n,)([\]\d!%|!#$%&*+,.-/:;<=>?@^_`{|}~?]','',str(i))).strip().lower() for i in training_data]

In [0]:
train_texts_shuffle = [i.split() for i in train_texts]
train_texts_shuffle2 = []
for i in train_texts_shuffle:
    for j in i:
        train_texts_shuffle2.append(j)
# train_texts_shuffle.extend(train_texts_shuffle2)

In [0]:
for k,i in enumerate(train_texts_shuffle2):
    if len(i) == 1:
        del train_texts_shuffle2[k]

In [0]:
train_texts = train_texts_shuffle2

In [0]:
train_texts_output = [1 for l in train_texts]

In [0]:
import random
import copy
train_texts_temp = copy.deepcopy(train_texts_shuffle2)
for i in range(5000):
    rand = random.randint(0,len(train_texts_temp)-1)
    rand2 = random.randint(0,len(train_texts_temp)-1)
    train_texts.append([train_texts_temp[rand]+train_texts_temp[rand2]])
    train_texts_output.append(1)

In [0]:
import random
# train_texts_data = []
for i in range(5000):
    rand = random.randint(0,len(train_texts_temp)-1)
    rand2 = random.randint(0,len(train_texts_temp)-1)
    rand3 = random.randint(0,len(train_texts_temp)-1)
    train_texts.append([train_texts_temp[rand]+train_texts_temp[rand2]+train_texts_temp[rand3]])
    train_texts_output.append(1)

In [0]:
print(len(train_texts),len(train_texts_output))

10618 10618


In [0]:
import numpy as np

CHAR_CODE_START = 1
CHAR_CODE_PADDING = 0


def build_characters_encoding(names):
    """
    :param names: list of strings
    :return: (encoding, decoding, count)
    """
    count = 3
    encoding = {}
    decoding = {1: 'START',2:'UNK'}
    for c in set([c for name in names for c in name]):
        encoding[c] = count
        decoding[count] = c
        count += 1
    return encoding, decoding, count


def transform(encoding, data, vector_size=20):
    """
    :param encoding: encoding dict built by build_characters_encoding()
    :param data: list of strings
    :param vector_size: size of each encoded vector
    """
    transformed_data = np.zeros(shape=(len(data), vector_size))
    for i in range(len(data)):
        for j in range(min(len(data[i]), vector_size)):
            transformed_data[i][j] = encoding[data[i][j]]
    return transformed_data


def decode(decoding, vector):
    """
    :param decoding: decoding dict built by build_characters_encoding()
    :param vector: an encoded vector
    """
    text = ''
    for i in vector:
        if i == 0:
            break
        text += decoding[i]
    return text


In [0]:
temp=['abcdefghijklmnopqrstuvwxyz1234567890!@#$%^&*(){}[]:<>?/,.~`']

In [0]:
input_encoding, input_decoding, input_dict_size = build_characters_encoding(temp)

In [0]:
letters = ['a','b','c','d','e','f','g','h','i','j','k','l','m',
           'n','o','p','q','r','s','t','u','v','w','x','y','z','1','2','3','4','5','6','7','8','9','0','!','@','#','$','%'
          '^','&','*','(',')','<','>',',','/','\\','|',':',';','~','`','+','-','.']

def noise_maker(sentence, threshold):
    '''Relocate, remove, or add characters to create spelling mistakes'''
    
    noisy_sentence = []
    i = 0
    while i < len(sentence):
        random = np.random.uniform(0,1,1)
        # Most characters will be correct since the threshold value is high
        if random < threshold:
            noisy_sentence.append(sentence[i])
        else:
            new_random = np.random.uniform(0,1,1)
            # ~33% chance characters will swap locations
            if new_random > 0.67:
                if i == (len(sentence) - 1):
                    # If last character in sentence, it will not be typed
                    continue
                else:
                    # if any other character, swap order with following character
                    noisy_sentence.append(sentence[i+1])
                    noisy_sentence.append(sentence[i])
                    i += 1
            # ~33% chance an extra lower case letter will be added to the sentence
            elif new_random < 0.33:
                random_letter = np.random.choice(letters, 1)[0]
                noisy_sentence.append(random_letter)
                noisy_sentence.append(sentence[i])
            # ~33% chance a character will not be typed
            else:
                pass     
        i += 1
    return noisy_sentence

In [0]:
input_encoding['a']

15

In [0]:
# Check to ensure noise_maker is making mistakes correctly.
threshold = 0.9
for sentence in ['Check to hindu noise_maker is making mistakes correctly']:
    print(sentence)
    d = noise_maker(sentence, threshold)
    print()
print(''.join(map(str,d)))

Check to hindu noise_maker is making mistakes correctly

Check to hindun oise_maker is making mistakes correctly


In [0]:
import copy
training_output = copy.deepcopy(train_texts_output)
training_input =  copy.deepcopy(train_texts)

In [0]:
for k in [0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95]:
    for i in range(7):
        for m,sentence in enumerate(train_texts):
            threshold = k
            training_input.append(''.join(map(str,noise_maker(sentence, threshold))))
            training_output.append(1)



In [0]:
len(training_input)

753878

In [0]:
len(training_output)

753878

In [0]:
import pandas as pd
data_unknown = pd.read_csv('joined_titles.csv')
data_unknown = data_unknown['Unschooling']
import unidecode 
data_unknown_ = [unidecode.unidecode(s).lower()  for s in data_unknown]

# training_input.extend(data_unknown_)
for i in range(5000):
    rand = random.randint(0,len(train_texts)-1)
    rand2 = random.randint(0,len(data_unknown_)-1)
    rand3 = random.randint(0,len(train_texts)-1)
    #+''.join(map(str,noise_maker(train_texts[rand3], 0.7)))
    training_input.append([data_unknown_[rand2]+''.join(map(str,noise_maker(train_texts[rand], 0.8)))+''.join(map(str,noise_maker(train_texts[rand3], 0.8)))])
    training_output.append(2)
#     training_output_2.append(train_texts[rand])
#     training_output_3.append("UNK")
            
#     training_output.append('UNK')

In [0]:
import pandas as pd
data_unknown = pd.read_csv('joined_titles.csv')
data_unknown = data_unknown['Unschooling']
import unidecode 
data_unknown_ = [unidecode.unidecode(s).lower()  for s in data_unknown]

# training_input.extend(data_unknown_)
for i in range(5000):
    rand = random.randint(0,len(train_texts)-1)
    rand2 = random.randint(0,len(data_unknown_)-1)
#     rand3 = random.randint(0,len(train_texts)-1)
    #+''.join(map(str,noise_maker(train_texts[rand3], 0.7)))
    training_input.append([''.join(map(str,noise_maker(train_texts[rand], 0.8)))+data_unknown_[rand2]])
    training_output.append(2)
#     training_output_2.append(train_texts[rand])
#     training_output_3.append("UNK")
            
#     training_output.append('UNK')

In [0]:
import pandas as pd
data_unknown = pd.read_csv('joined_titles.csv')
data_unknown = data_unknown['Unschooling']
import unidecode 
data_unknown_ = [unidecode.unidecode(s).lower()  for s in data_unknown]

# training_input.extend(data_unknown_)
for i in range(5000):
    rand = random.randint(0,len(train_texts)-1)
    rand2 = random.randint(0,len(data_unknown_)-1)
#     rand3 = random.randint(0,len(train_texts)-1)
    #+''.join(map(str,noise_maker(train_texts[rand3], 0.7)))
    training_input.append([data_unknown_[rand2]+train_texts[rand][0]])
    training_output.append(2)
#     training_output_2.append(train_texts[rand])
#     training_output_3.append("UNK")
            
#     training_output.append('UNK')

In [0]:
import pandas as pd
data_unknown = pd.read_csv('joined_titles.csv')
data_unknown = data_unknown['Unschooling']
import unidecode 
data_unknown_ = [unidecode.unidecode(s).lower()  for s in data_unknown]

# training_input.extend(data_unknown_)
for i in range(5000):
    rand = random.randint(0,len(train_texts)-1)
    rand2 = random.randint(0,len(data_unknown_)-1)
    rand3 = random.randint(0,len(train_texts)-1)
    #+''.join(map(str,noise_maker(train_texts[rand3], 0.7)))
    training_input.append([train_texts[rand][0]+data_unknown_[rand2]+train_texts[rand3][0]])
    training_output.append(2)
#     training_output_2.append(train_texts[rand])
#     training_output_3.append("UNK")
            
#     training_output.append('UNK')

In [0]:
with open('check_unk_know_classification_dataset','wb') as f:
#     training_input, training_output = pickle.load(f)
    pickle.dump([training_input,training_output],f)

In [0]:
import pandas as pd
data_unknown = pd.read_csv('joined_titles.csv')
data_unknown = data_unknown['Unschooling']
import unidecode 
data_unknown_ = [unidecode.unidecode(s).lower()  for s in data_unknown]
training_input.extend(data_unknown_)
for i in  data_unknown_:
    training_output.append(2)

In [0]:
for k,i in enumerate(training_input):
    if isinstance(i,list):
        training_input[k] = i[0]

In [0]:
training_input

['where',
 'is',
 'your',
 'showroom',
 'located',
 'is',
 'there',
 'any',
 'showrooms',
 'in',
 'chennai',
 'can',
 'you',
 'share',
 'me',
 'your',
 'showroom',
 'address',
 'how',
 'will',
 'find',
 'you',
 'in',
 'chennai',
 'can',
 'get',
 'your',
 'sales',
 'executive',
 'phone',
 'number',
 'want',
 'sales',
 'executive',
 'phone',
 'number',
 'how',
 'can',
 'contact',
 'dream',
 'cards',
 'contact',
 'number',
 'for',
 'dream',
 'cards',
 'any',
 'whatsapp',
 'number',
 'for',
 'dream',
 'cards',
 'phone',
 'number',
 'please',
 'iam',
 'looking',
 'for',
 'wedding',
 'cards',
 'iam',
 'looking',
 'for',
 'hindu',
 'wedding',
 'cards',
 'iam',
 'looking',
 'for',
 'birthday',
 'cards',
 'iam',
 'looking',
 'for',
 'wedding',
 'card',
 'iam',
 'looking',
 'for',
 'wedding',
 'card',
 'iam',
 'looking',
 'for',
 'birthday',
 'cards',
 'iam',
 'looking',
 'for',
 'hindu',
 'wedding',
 'card',
 'iam',
 'looking',
 'for',
 'muslim',
 'wedding',
 'cards',
 'iam',
 'looking',
 'for'